## data_preprocessing1로 끝나는 파일에서 살펴본 내용을 정리하여 제가 원하는 데이터 셋으로 전처리하여 새로운 csv 파일로 저장하는 과정만 따로 포함하고 있습니다
## analysis & conclusion에서 오류/이상치들을 파악한뒤 다시 정리하여 또 새로운 파일로 export 한 과정을 포함하고 있습니다

In [1]:
import pandas as pd
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Project_section2/train.csv').fillna('NaN')
# train = train[train.game_id < 10000]
# train.to_csv('/content/drive/MyDrive/Colab Notebooks/train9999.csv',sep=',',index=False)
# train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/gamedata.csv')
train

,game_id,winner,time,player,species,event,event_contents
0,0,1,0.00,0,T,Camera,"at (145.25, 21.5078125)"
1,0,1,0.00,1,T,Camera,"at (22.75, 147.0078125)"
2,0,1,0.02,0,T,Selection,['OrbitalCommand [3080001]']
3,0,1,0.02,0,T,Ability,(1360) - TrainSCV
4,0,1,0.14,0,T,Camera,"at (142.99609375, 24.50390625)"
...,...,...,...,...,...,...,...
67091771,38871,0,8.51,0,Z,Camera,"at (139.578125, 62.58203125)"
67091772,38871,0,8.52,1,T,GetControlGroup,NaN
67091773,38871,0,8.52,0,Z,Camera,"at (122.42578125, 45.4296875)"
67091774,38871,0,8.52,0,Z,Camera,"at (122.42578125, 43.25390625)"


In [ ]:
train[(train.species=='P') & (train.event_contents.str.contains('Train')) &(~train.event_contents.str.contains('TrainProbe')) ]

,game_id,winner,time,player,species,event,event_contents
1894,1,1,3.08,0,P,Ability,(1586) - TrainAdept
1897,1,1,3.10,0,P,Ability,(2720) - TrainMothershipCore
2263,1,1,4.50,0,P,Ability,(15C0) - TrainWarpPrism
2833,1,1,7.17,0,P,Ability,(15D2) - TrainDisruptor
2891,1,1,7.26,0,P,Ability,(1AC6) - TrainAdept; Location: (121.0095214843...
...,...,...,...,...,...,...,...
67088127,38870,1,3.29,0,P,Ability,(1581) - TrainStalker
67088160,38870,1,3.40,1,P,Ability,(1581) - TrainStalker
67088210,38870,1,3.53,1,P,Ability,(1581) - TrainStalker
67088335,38870,1,4.30,1,P,Ability,(15A0) - TrainPhoenix


In [ ]:
train[train['game_id']==18139]

,game_id,winner,time,player,species,event,event_contents
31254949,18139,0,0.00,0,T,Camera,"at (152.25, 142.0078125)"
31254950,18139,0,0.00,1,Z,Camera,"at (31.75, 22.5078125)"
31254951,18139,0,0.01,0,T,Selection,['CommandCenter [3100001]']
31254952,18139,0,0.01,0,T,Ability,(1360) - TrainSCV
31254953,18139,0,0.01,0,T,Right Click,"Location: (156.25, 142.182373046875, 49119)"
...,...,...,...,...,...,...,...
31255056,18139,0,0.20,0,T,Camera,"at (32.109375, 70.48828125)"
31255057,18139,0,0.20,0,T,Camera,"at (32.109375, 71.0234375)"
31255058,18139,0,0.20,1,Z,Selection,"['Egg [34C0001]', 'Larva [3500001]']"
31255059,18139,0,0.20,0,T,Right Click,"Location: (28.2080078125, 77.296142578125, 40926)"


In [ ]:
times = train.groupby('game_id', as_index=False).agg({
    'time':'last'
})
times.to_csv('/content/drive/MyDrive/Colab Notebooks/Project_section2/times.csv',sep=',',index=False)

In [ ]:
# 같은 game_id를 공유하는 row들을 player 별로 정보를 나누어 player별 실행한 행위 별로 칼럼을 재생성
# 예) player_0_Camera는 player 0이 한 게임동안 "event : Camera" 를 실행한 횟수

events = ['Camera', 'Selection', 'Right Click', 'Ability', 'GetControlGroup', 'SetControlGroup', 'AddToControlGroup']

# game_id = 0 ~ 38872 이고 원 데이터가 6000만 row를 가지고 있는 대용량 데이터여서 실행시간을 줄이고
# 에러를 바로바로 확인할 수 있도록 game_id를 약 만개단위로 잘라서 아래셀 for문 range() 에 넣었음

# initial dataframe은 후에 계속 생기는 row들을 append 해주기 위해 처음 만들어주는 데이터 프레임
# game_id 는 0부터 38872(train),  38873~55658은 test 이나 test는 원래 데이콘 제출용이기 때문에 
# 타겟 칼럼이 없어서 혼자서 데이터와 모델을 실행해볼수 없어서 진행하다 중단

ex0 = train[(train.game_id == 38872 ) & (train.player== 0 )]
initial = ex0.groupby('player',as_index=False).agg({
    'game_id': 'first',
    'winner': 'first', 
    'time':'last', 
    'species' : 'first', 
    'event' : ','.join,
    'event_contents':'first'
    # 'Platform':','.join,
    })
b = pd.Series(initial.event[0].split(',')).value_counts()
initial['player_0_species'] = initial['species']
initial = initial.drop(['player','event','event_contents','species'],axis=1)
for i in events:
  if i not in b.index:
    initial[f'player_{0}_'+i] = 0  
    continue
  initial[f'player_{0}_'+i] = b[i]

ex0 = train[(train.game_id == 38872 ) & (train.player== 1 )]
a = ex0.groupby('player',as_index=False).agg({
    'game_id': 'first',
    'winner': 'first', 
    'time':'last', 
    'species' : 'first', 
    'event' : ','.join,
    'event_contents':'first'
    # 'Platform':','.join,
    })
a
b = pd.Series(a.event[0].split(',')).value_counts()
initial['player_1_species'] = a['species']

for i in events:
  if i not in b.index:
    initial[f'player_{1}_'+i] = 0
    continue  
  initial[f'player_{1}_'+i] = b[i]

initial.T

In [ ]:
for id in range(38873,55659):
  print(id)
  ex0 = train[(train.game_id == id ) & (train.player== 0 )]
  tmp0 = ex0.groupby('player',as_index=False).agg({
      'game_id': 'first',
      'winner': 'first', 
      'time':'last', 
      'species' : 'first', 
      'event' : ','.join,
      'event_contents':'first'
      # 'Platform':','.join,
      })
  b = pd.Series(tmp0.event[0].split(',')).value_counts()
  tmp0['player_0_species'] = tmp0['species']
  tmp0 = tmp0.drop(['player','event','event_contents','species'],axis=1)
  for i in events:
    if i not in b.index:
      tmp0[f'player_{0}_'+i] = 0  
      continue
    tmp0[f'player_{0}_'+i] = b[i]

  ex0 = train[(train.game_id == id ) & (train.player== 1 )]
  tmp1 = ex0.groupby('player',as_index=False).agg({
      'game_id': 'first',
      'winner': 'first', 
      'time':'last', 
      'species' : 'first', 
      'event' : ','.join,
      'event_contents':'first'
      # 'Platform':','.join,
      })
  b = pd.Series(tmp1.event[0].split(',')).value_counts()
  tmp0['player_1_species'] = tmp1['species']

  for i in events:
    if i not in b.index:
      tmp0[f'player_{1}_'+i] = 0
      continue  
    tmp0[f'player_{1}_'+i] = b[i]
  initial = initial.append(tmp0.iloc[0])

initial.to_csv('/content/drive/MyDrive/Colab Notebooks/project2_data_sc2_test.csv',sep=',',index=False)

In [ ]:
# train = train.fillna('NaN')
train[train.event_contents.str.contains('Recall')]

,game_id,winner,time,player,species,event,event_contents
226119,137,0,6.23,1,P,Ability,(2B00) - MassRecallMothershipCore; Target: Nex...
226212,137,0,6.51,1,P,Ability,(2B00) - MassRecallMothershipCore; Target: Nex...
237428,146,1,7.33,1,P,Ability,(2B00) - MassRecallMothershipCore; Target: Nex...
494932,289,1,6.54,1,P,Ability,(2B00) - MassRecallMothershipCore; Target: Nex...
696293,407,1,6.23,0,P,Ability,(2B00) - MassRecallMothershipCore; Target: Nex...
...,...,...,...,...,...,...,...
65211276,37790,0,9.48,0,P,Ability,(2B00) - MassRecallMothershipCore; Target: Nex...
65429540,37906,1,5.30,1,P,Ability,(2B00) - MassRecallMothershipCore; Target: Nex...
65857088,38147,1,5.10,0,P,Ability,(2B00) - MassRecallMothershipCore; Target: Nex...
66197345,38350,1,6.06,0,P,Ability,(2B00) - MassRecallMothershipCore; Target: Nex...


In [ ]:
gamedata = train[train.game_id > 30000]
gamedata.to_csv('/content/drive/MyDrive/Colab Notebooks/gamedata.csv',sep=',',index=False)

In [ ]:
games100 = train[train.game_id < 100]
games100.to_csv('/content/drive/MyDrive/Colab Notebooks/games100.csv',sep=',',index=False)
games300 = train[train.game_id < 300]
games300.to_csv('/content/drive/MyDrive/Colab Notebooks/games300.csv',sep=',',index=False)



In [ ]:
gameid0 = train[train.game_id==0]
gameid0.to_csv('gameid0.csv', sep=',', index=False)


In [ ]:
gameid0[gameid0.player == 0 ]

,game_id,winner,time,player,species,event,event_contents
0,0,1,0.00,0,T,Camera,"at (145.25, 21.5078125)"
2,0,1,0.02,0,T,Selection,['OrbitalCommand [3080001]']
3,0,1,0.02,0,T,Ability,(1360) - TrainSCV
4,0,1,0.14,0,T,Camera,"at (142.99609375, 24.50390625)"
5,0,1,0.14,0,T,Camera,"at (142.5078125, 24.98828125)"
...,...,...,...,...,...,...,...
1134,0,1,7.22,0,T,Camera,"at (144.36328125, 42.44921875)"
1135,0,1,7.22,0,T,Camera,"at (144.36328125, 44.93359375)"
1136,0,1,7.22,0,T,Camera,"at (144.36328125, 45.75390625)"
1138,0,1,7.24,0,T,Right Click,"Target: None [00CC0001]; Location: (155.0, 54...."
